# 1. Symplectic representation

## 1.1. On Humphries generators

### Sp_repr class and functions

In [1]:
%display latex

In [2]:
class Sp_repr:
    #--- 2x2 ---
    I = identity_matrix(2)
    O = zero_matrix(ZZ,2)
    J = matrix([[0,1],[-1,0]])
    #---
    L = matrix([[1,0],[1,1]])
    K = matrix([[0,-1],[0,0]])
    #--- 4x4 ---
    J4 = block_matrix([[J, O], [O, J]])
    # J4 = J4_bm.subs([(J, evJ)]).as_explicit()
    #---
    CHR_2_MTX = {'a': block_matrix([[L.inverse(), O], [O, I]]),
                 'b': block_matrix([[L.T, K], [K, L.T]]),
                 'c': block_matrix([[I, O], [O, L.inverse()]]),
                 'd': block_matrix([[I, O], [O, L.T]]),
                 'f': block_matrix([[L.T, O], [O,I]])}
    #---
    
    def __init__(self, l:str):
        self.loop = l
        self.matrix = self.CHR_2_MTX.get(l) if l.islower() else self.CHR_2_MTX.get(l.lower()).inverse()

In [3]:
def is_Sp(M):
    J = Sp_repr.J4
    return M.transpose()*J*M == J

list(map(lambda l: is_Sp(Sp_repr(l).matrix), ['a','b','c','d','f', 'A'])) 

[True, True, True, True, True, True]

## 1.2. The symplectic images of Abcd and abcD

In [4]:
a,b,c,d,f = tuple(map(lambda l: Sp_repr(l).matrix, ['a','b','c','d','f']))
J4 = Sp_repr.J4

a,b,c,d,f, J4 

(
[ 1  0| 0  0]  [ 1  1| 0 -1]  [ 1  0| 0  0]  [1 0|0 0]  [1 1|0 0]
[-1  1| 0  0]  [ 0  1| 0  0]  [ 0  1| 0  0]  [0 1|0 0]  [0 1|0 0]
[-----+-----]  [-----+-----]  [-----+-----]  [---+---]  [---+---]
[ 0  0| 1  0]  [ 0 -1| 1  1]  [ 0  0| 1  0]  [0 0|1 1]  [0 0|1 0]
[ 0  0| 0  1], [ 0  0| 0  1], [ 0  0|-1  1], [0 0|0 1], [0 0|0 1],

[ 0  1| 0  0]
[-1  0| 0  0]
[-----+-----]
[ 0  0| 0  1]
[ 0  0|-1  0]
)

In [5]:
A, B, C, D, F = (k.inverse() for k in [a,b,c,d,f])
A,B,C,D,F

(
[1 0 0 0]  [ 1 -1  0  1]  [1 0 0 0]  [ 1  0  0  0]  [ 1 -1  0  0]
[1 1 0 0]  [ 0  1  0  0]  [0 1 0 0]  [ 0  1  0  0]  [ 0  1  0  0]
[0 0 1 0]  [ 0  1  1 -1]  [0 0 1 0]  [ 0  0  1 -1]  [ 0  0  1  0]
[0 0 0 1], [ 0  0  0  1], [0 0 1 1], [ 0  0  0  1], [ 0  0  0  1]
)

In [6]:
M = a*b*c*d
display(M)
N = a*b*a*C*D*f
display(N)

list(map(is_Sp, [M, N]))

[ 1  1  1  0]
[-1  0 -1  0]
[ 0 -1  0  1]
[ 0  0 -1  0]

[ 0  1 -1  0]
[-1 -1  1  0]
[ 1  0  2 -1]
[ 0  0  1  0]

[True, True]

### The charactoristic polynomial of M (and N)

In [7]:
R.<a> = PolynomialRing(ZZ)
cp_M = M.charpoly('t'); display(cp_M, cp_M.factor())
cp_N = N.charpoly('t'); #display(cp_N)

print(f"cp_M == cp_N ?   --> {cp_M == cp_N}")

t^4 - t^3 + t^2 - t + 1

t^4 - t^3 + t^2 - t + 1

cp_M == cp_N ?   --> True


# 2. The ideal for fibered knots

In [8]:
K.<a> = NumberField(cp_M)
OK = K.ring_of_integers()
print("整数基:", OK.basis())
print("類数:", OK.class_number())

U = UnitGroup(K)
print(U.gens_values())

G = K.galois_group()
print(G)

整数基: [1, a, a^2, a^3]
類数: 1
[a, -a + 1]
Galois group 4T1 (4) with order 4 of t^4 - t^3 + t^2 - t + 1


In [9]:
def ev_wrt_a(X):
    X_K = X.change_ring(K)
    evlist = X_K.eigenvectors_right();
    v = [v[1][0] for v in evlist if v[0] == a][0]
    if X_K*v == a*v:
        return v
    else:
        return None

### Eigenvectors of M and N with respect to the eigen value $a$

In [10]:
v, w = [ev_wrt_a(X) for X in (M,N)]
display(v, w)
print("------- Check -------")
print(M*v == a*v, N*w == a*w)

(1, a^3 + a - 1, -a^3, a^2)

(1, a^3 - a^2 + a - 2, a^3 - a^2 - 2, 2*a^3 - a^2 + a - 2)

------- Check -------
True True


### Ideals of M and N

In [11]:
# The integral ideal (v) is equal to ZZ[a]

T = matrix([[1,0,0,0],[1,1,1,0],[0,0,0,1],[0,0,-1,0]]); display(T, T*v)
X = T; display(X)

[ 1  0  0  0]
[ 1  1  1  0]
[ 0  0  0  1]
[ 0  0 -1  0]

(1, a, a^2, a^3)

[ 1  0  0  0]
[ 1  1  1  0]
[ 0  0  0  1]
[ 0  0 -1  0]

In [12]:
# The integral ideal (w) is equal to ZZ[a]
display(T)
T = matrix([[1,0,0,0],[0,1,-1,0],[0,0,0,1],[0,0,1,0]]); display(T*w)
T2 = matrix([[1,0,0,0],[0,1,0,0],[-2,-1,1,-2],[0,0,0,1]]); display(T2*(T*w))
T3 = matrix([[1,0,0,0],[0,1,0,0],[0,0,1,0],[2,0,1,1]]); display(T3*(T2*(T*w)))
Y = T3*T2*T; display(Y)

[ 1  0  0  0]
[ 1  1  1  0]
[ 0  0  0  1]
[ 0  0 -1  0]

(1, a, 2*a^3 - a^2 + a - 2, a^3 - a^2 - 2)

(1, a, a^2, a^3 - a^2 - 2)

(1, a, a^2, a^3)

[ 1  0  0  0]
[ 0  1 -1  0]
[-2 -1 -1  1]
[ 0 -1  0  1]

In [13]:
S = Y.inverse()*X
display(S, S.inverse())

print(f"{(S*v == w) = }")
display(S*v, w)
print(f"{is_Sp(S)=}")

[ 1  0  0  0]
[-1  1  0 -1]
[-2  0 -1 -1]
[-1  1 -1 -1]

[ 1  0  0  0]
[-1  0 -1  1]
[ 0  1  0 -1]
[-2 -1 -1  1]

(S*v == w) = True


(1, a^3 - a^2 + a - 2, a^3 - a^2 - 2, 2*a^3 - a^2 + a - 2)

(1, a^3 - a^2 + a - 2, a^3 - a^2 - 2, 2*a^3 - a^2 + a - 2)

is_Sp(S)=False


### Scratch

In [ ]:
#print(v, type(v))
display(a+1, (a+1).subs({a: 1/a}))
display(v, v.subs({a: 1/a}), J4*(v.subs({a: 1/a})))
# v.dot_product(J4*v)

In [26]:
tv = vector([1, 1/a^3+1/a-1, -1/a^3, 1/a^2])
display(tv)
alpha = v.dot_product(J4*tv)
display(alpha)

(1, -a^3 - a, a^2, -a^3)

-3*a^3 + a^2 - 4*a + 2

In [27]:
(a-2)*(1/a-2)

2*a^3 - 2*a^2 + 3

In [ ]:
# 双一次形式を関数として定義
def bilinear_form(x, y, A):
    """
    行列Aで定義される双一次形式b(x, y) = x^T * A * y を計算する
    """
    return x.dot_product(A * y)

# 同じ行列とベクトルを使って計算
A = matrix(QQ, [[1, 2, 3], [4, 5, 6], [7, 8, 9]])
x = vector(QQ, [1, 0, 1])
y = vector(QQ, [0, 1, 1])

# 関数を呼び出す
result_value = bilinear_form(x, y, A)[0]
print(result_value)